## system configuration

In [ ]:
from cobra.io import read_sbml_model
model = read_sbml_model('iNF517.xml')

In [ ]:
model.optimize()

In [ ]:
%%time
copy_of_model = model.copy()

In [ ]:
%%time
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

In [ ]:
%%time
with model:
    for gene in model.genes:
        gene.knock_out()

In [7]:
with model:
    model.reactions.PGK.bounds = 0, 0
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.bounds)

PGK's bounds inside the with statement
0 (0, 0)
Mutant growth rate:  -3.697962547581411e-18
PGK's bounds outside the with statement
(-1000.0, 1000.0)


/opt/conda/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [8]:
%%time
solution = model.optimize()
solution.fluxes['ATPM']

CPU times: user 14.9 ms, sys: 379 µs, total: 15.3 ms
Wall time: 32 ms


0.0

In [9]:
%%time
model.slim_optimize()
model.reactions.ATPM.flux

CPU times: user 690 µs, sys: 88 µs, total: 778 µs
Wall time: 785 µs


0.0

In [10]:
model.medium

{'EX_4abz_e': 0.00999,
 'EX_ade_e': 0.01014,
 'EX_ala__L_e': 0.04,
 'EX_arg__L_e': 0.02,
 'EX_asp__L_e': 0.01,
 'EX_co2_e': 1000.0,
 'EX_cys__L_e': 0.02,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc__D_e': 2.12,
 'EX_glu__L_e': 0.05,
 'EX_gly_e': 0.04,
 'EX_gua_e': 0.00906,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_his__L_e': 0.01,
 'EX_ile__L_e': 0.05,
 'EX_ins_e': 0.00255,
 'EX_leu__L_e': 0.06,
 'EX_lys__L_e': 0.04,
 'EX_met__L_e': 0.01,
 'EX_mn2_e': 1000.0,
 'EX_nac_e': 0.00111,
 'EX_nh4_e': 0.59,
 'EX_orot_e': 0.00439,
 'EX_phe__L_e': 0.02,
 'EX_pi_e': 1000.0,
 'EX_pnto__R_e': 0.00062,
 'EX_ribflv_e': 0.00036,
 'EX_ser__L_e': 0.11,
 'EX_thm_e': 0.00041,
 'EX_thr__L_e': 0.06,
 'EX_thymd_e': 0.00283,
 'EX_ura_e': 0.01222,
 'EX_val__L_e': 0.05,
 'EX_xan_e': 0.00901,
 'EX_zn2_e': 1000.0}

In [11]:
solution = model.optimize()

In [12]:
solution.fluxes

ACALDt       0.000000
ACMANApts    0.000000
ACTNdiff     0.000000
ACt2r       -0.919307
ADEt2        0.004750
               ...   
MCMAT8       0.003012
METDabc      0.000000
PSERT        0.000000
BGLA1        0.000000
SUCpts       0.000000
Name: fluxes, Length: 754, dtype: float64

## medium with glucose

In [13]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 2.12
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])

1.02


with 2.12 mmol / gDW h glucose, the yiled of ethanol is 1.02 mmol / gDW

In [14]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_lcts_e'] = 1
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

0.07461055952589815


medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_lcts_e'] = 1
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

## medium without glucose

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

This result showed that L.lactics can not grow and produce ethanol without glucose

## Minimal media

In [ ]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

## medium with glucose and other carbon source

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_lcts_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    medium = model.medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_lcts_e'] = 1
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_malt_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    medium = model.medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_malt_e'] = 1
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_fru_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    medium = model.medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_fru_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_sucr_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    medium = model.medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_sucr_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_gal_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    medium = model.medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_gal_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_glyc_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    medium = model.medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1
    medium['EX_glyc_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

## in anaerobic condition

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 2.12
    medium['EX_o2_e'] = 0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    medium = model.medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 2.12
    medium['EX_o2_e'] = 0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

## in aerobic condition

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 2.12
    medium['EX_o2_e'] = 5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    medium = model.medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 2.12
    medium['EX_o2_e'] = 5
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

## Minimal media

In [1]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

NameError: name 'model' is not defined

In [ ]:
max_growth

In [ ]:
print 
1+1